In [1]:
from pylibCZIrw import czi as pyczi
import json
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
from numpy import ma
import os, sys
from tqdm import tqdm
from tqdm.contrib import itertools as it
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap
from cellpose import utils
import cv2
from scipy import ndimage
from pathlib import Path
import pandas as pd
from skimage import io, exposure


# show the used python env
print("Using:", sys.executable)

Using: /home/chrism/miniconda3/envs/cellpose3/bin/python


In [4]:
# nuclei extration

what = "test"

if what == "train":
    data_path = Path('/home/chrism/datasets/dapidl/train/tcell')
else:
    data_path = Path('/home/chrism/datasets/dapidl/test/tcell')
    

border = 2

area_pixel = 0.512 * 0.512
nuclei_path = data_path / "nuclei"
cells_path = data_path / "cells"
czi_path = data_path / "czi"
exclude_path = data_path / "exclude"
debug_path = data_path / f"debug_border_{border}"
classification_path = data_path / f"classification_border_{border}"

results = []

debug = False


colors_blue = [(0, 0, 0), (0, 0, 1)]
cm_blue = LinearSegmentedColormap.from_list("Custom", colors_blue, N=255)

colors_red = [(0, 0, 0), (1, 0, 0)]
cm_red = LinearSegmentedColormap.from_list("Custom", colors_red, N=255)

colors_green = [(0, 0, 0), (0, 1, 0)] 
cm_green = LinearSegmentedColormap.from_list("Custom", colors_green, N=255)

colors_white = [(0, 0, 0), (1, 1, 1)] 
cm_white = LinearSegmentedColormap.from_list("Custom", colors_white, N=255)

for cells_masks_file in cells_path.glob("**/*.npy"):
    print(cells_masks_file)
    slide = cells_masks_file.parent.parts[-1]
    group = cells_masks_file.parent.parts[-2]
    
    exclude_masks = exclude_path / cells_masks_file.name
    if not exclude_masks.exists():
        print(f"{exclude_masks} does not exist. Skipping ...")
        continue
        
    czi_file_name = cells_masks_file.name[0:-15] + '.czi'
    czi_file = czi_path / czi_file_name
    
    if not czi_file.exists():
        print(f"{czi_file} does not exist. Skipping ...")
        continue
        
    cells_masks = cells_path / cells_masks_file.name
    if not cells_masks.exists():
        print(f"{cells_masks} does not exist. Skipping ...")
        continue
    
    nuclei_masks_file = nuclei_path / cells_masks_file.name
    if not nuclei_masks_file.exists():
        print(f"{nuclei_masks_file} does not exist. Skipping ...")
        continue

    nuclei_seg = np.load(nuclei_masks_file, allow_pickle=True).item()
    nuclei_masks = nuclei_seg["masks"]
    num_nuclei = nuclei_masks.max()
    
    exclude_seg = np.load(exclude_masks, allow_pickle=True).item()
    exclude_masks = exclude_seg["masks"] > 0

    cells_seg = np.load(cells_masks, allow_pickle=True).item()
    cells_masks = cells_seg["masks"]
    num_cells = cells_masks.max()
    
    num_pixels = exclude_masks.shape[0] * exclude_masks.shape[1] - np.count_nonzero(exclude_masks)
    area = num_pixels * area_pixel
    # print(area)
    nuclei_per_um2 = num_nuclei / area
    # print(cells_per_um2)

    with pyczi.open_czi(str(czi_file)) as czidoc:
        ch0 = czidoc.read(plane={'C': 0})
        dapi = ch0[..., 0]
        ch1 = czidoc.read(plane={'C': 1})
        mrc1 = ch1[..., 0]
        ch2 = czidoc.read(plane={'C': 2})
        vwf = ch2[..., 0]
        # ch3 = czidoc.read(plane={'C': 3})
        # f480 = ch3[..., 0]

    if debug:
        # d_slices = utils.find_objects(masks)
        # d_mrc1 = mrc1.copy()
        # for i,si in enumerate(slices):
        #     if si is not None:
        #         sr,sc = si
        #         mask = (masks[sr, sc] == (i+1)).astype(np.uint8)
        #         contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        #         ax.contours(
        #         d_mrc1 = cv2.drawContours(d_mrc1, contours, -1, (0, 255, 0), 2)
                
        plt.ioff()
        fig, ax = plt.subplots(figsize=(13, 10))
        ax.imshow(mrc1, cmap=cm_red)

    # cells_slices = utils.find_objects(cells_masks)
    # cells_centers = ndimage.center_of_mass(mrc1, cells_masks, index=np.arange(1, num_cells + 1))

    # for i, (si, (row, col)) in enumerate(zip(cells_slices, cells_centers)): 
    #     if np.any((cells_masks == i+1) & exclude_masks):
    #         print(f"Cell {i+1} intersecting exclude region. Skipping ...")
    #         continue
    #     if si is not None:
    #         sr, sc = si
    #         cell_mask = cells_masks[sr, sc] != (i+1)
    #         dapi_ma = ma.masked_array(dapi[sr, sc], mask)

    
    slices = utils.find_objects(nuclei_masks)
    nuclei_centers = ndimage.center_of_mass(dapi, nuclei_masks, index=np.arange(1, num_nuclei + 1))

    for i, (si, (row, col)) in enumerate(zip(slices, nuclei_centers)): 
        if np.any((nuclei_masks == i+1) & exclude_masks):
            print(f"Nucleus {i+1} intersecting exclude region. Skipping ...")
            continue
        if np.any((nuclei_masks == i+1) & (cells_masks != 0)):
            nucleus_class = "tcell"
        else:
            nucleus_class = "other"
        if si is not None:
            sr, sc = si
            if sr.start - border >= 0:
                sr_start = sr.start - border
            else:
                sr_start = sr.start
                
            if sr.stop + border < 1040:
                sr_stop = sr.stop + border
            else:
                sr_stop = sr.stop
                
            if sc.start - border >= 0:
                sc_start = sc.start - border
            else:
                sc_start = sc.start
                
            if sc.stop + border < 1388:
                sc_stop = sc.stop + border
            else:
                sc_stop = sc.stop
                
            mask = nuclei_masks[sr_start:sr_stop, sc_start:sc_stop] != (i+1)
            dapi_ma = ma.masked_array(dapi[sr_start:sr_stop, sc_start:sc_stop], mask)
            # mrc1_ma = ma.masked_array(mrc1[sr, sc], mask)
            # vwf_ma = ma.masked_array(vwf[sr, sc], mask)
            # f480_ma = ma.masked_array(f480[sr, sc], mask)
            # mrc1_f480_ratio = mrc1_ma.sum() / f480_ma.sum()
            # f480_mrc1_ratio = f480_ma.sum() / mrc1_ma.sum()
            # print(f"MRC1 F4/80 ratio = {mrc1_f480_ratio}")
        
        # print(row, col)
        row = round(row)
        col = round(col)
        
        # save image

        im12 = dapi_ma.filled(0)
        # im8 = exposure.rescale_intensity(im12, in_range=(0, 2**12), out_range=np.uint8)
        im8 = exposure.rescale_intensity(dapi[sr, sc], in_range=(0, 2**12), out_range=np.uint8)
        png_file_name = f"{czi_file_name}_{i:0>6}.png"
        png_file_path = classification_path / nucleus_class / png_file_name
        if not png_file_path.parent.exists():
            png_file_path.parent.mkdir(parents=True, exist_ok=True)
        io.imsave(png_file_path, im8)
        results_row = {'czi': czi_file_name, 'nucleus_no': i, 'class': nucleus_class, 'filename': png_file_name, 'mean(dapi)': dapi_ma.mean()}
        results.append(results_row)

        if debug:
            ax.text(col, row, str(i), color="white")
            ax.scatter(col, row, c="yellow", marker=".")
            # d_mask = (cell_masks[sr, sc] == (i+1)).astype(np.uint8)
            # d_contours, _ = cv2.findContours(d_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            # d_contours, _ = cv2.findContours(d_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
            # d_contour = d_contours[0]
            
            # M = cv2.moments(d_contour)
            # x = int(M["m10"] / M["m00"])
            # y = int(M["m01"] / M["m00"])
            
            # xs = [v[0][0] - x for v in d_contour]
            # ys = [-(v[0][1] - y) for v in d_contour]
            
            # ax.plot(xs, ys, color="green")
            

    if debug:
        debug_file_name = nuclei_masks_file.name + ".png"
        debug_file = debug_path / debug_file_name
        debug_file.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(str(debug_file))
        plt.close(fig)
        plt.ion()
        
df = pd.DataFrame(results)
out_fn = f"results_nuclei_border_{border}.xlsx"
df.to_excel(data_path / out_fn)

/home/chrism/datasets/dapidl/test/tcell/cells/WT___CD8 1757 WT___Snap-147704_c1+2+3_seg.npy


/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000000.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000006.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000010.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000012.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/cl

Nucleus 429 intersecting exclude region. Skipping ...
Nucleus 448 intersecting exclude region. Skipping ...
Nucleus 457 intersecting exclude region. Skipping ...
Nucleus 487 intersecting exclude region. Skipping ...
Nucleus 512 intersecting exclude region. Skipping ...
Nucleus 518 intersecting exclude region. Skipping ...
Nucleus 536 intersecting exclude region. Skipping ...
Nucleus 538 intersecting exclude region. Skipping ...
Nucleus 563 intersecting exclude region. Skipping ...
Nucleus 569 intersecting exclude region. Skipping ...
Nucleus 597 intersecting exclude region. Skipping ...
Nucleus 606 intersecting exclude region. Skipping ...
Nucleus 621 intersecting exclude region. Skipping ...
Nucleus 652 intersecting exclude region. Skipping ...
Nucleus 654 intersecting exclude region. Skipping ...


/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000658.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000663.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000674.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147704.czi_000677.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/cl

Nucleus 694 intersecting exclude region. Skipping ...
Nucleus 695 intersecting exclude region. Skipping ...
Nucleus 697 intersecting exclude region. Skipping ...
Nucleus 709 intersecting exclude region. Skipping ...
Nucleus 727 intersecting exclude region. Skipping ...
Nucleus 739 intersecting exclude region. Skipping ...
Nucleus 743 intersecting exclude region. Skipping ...
Nucleus 751 intersecting exclude region. Skipping ...
Nucleus 763 intersecting exclude region. Skipping ...
Nucleus 782 intersecting exclude region. Skipping ...
Nucleus 794 intersecting exclude region. Skipping ...
Nucleus 806 intersecting exclude region. Skipping ...
Nucleus 810 intersecting exclude region. Skipping ...
Nucleus 813 intersecting exclude region. Skipping ...
Nucleus 829 intersecting exclude region. Skipping ...
Nucleus 831 intersecting exclude region. Skipping ...
Nucleus 837 intersecting exclude region. Skipping ...
Nucleus 845 intersecting exclude region. Skipping ...
Nucleus 857 intersecting exc

...

Nucleus 1092 intersecting exclude region. Skipping ...
Nucleus 1093 intersecting exclude region. Skipping ...
Nucleus 1111 intersecting exclude region. Skipping ...
Nucleus 1112 intersecting exclude region. Skipping ...
Nucleus 1122 intersecting exclude region. Skipping ...
Nucleus 1130 intersecting exclude region. Skipping ...
Nucleus 1169 intersecting exclude region. Skipping ...
Nucleus 1178 intersecting exclude region. Skipping ...
Nucleus 1200 intersecting exclude region. Skipping ...
Nucleus 1203 intersecting exclude region. Skipping ...
Nucleus 1205 intersecting exclude region. Skipping ...
Nucleus 1211 intersecting exclude region. Skipping ...
Nucleus 1216 intersecting exclude region. Skipping ...
Nucleus 1229 intersecting exclude region. Skipping ...
Nucleus 1236 intersecting exclude region. Skipping ...
Nucleus 1241 intersecting exclude region. Skipping ...
Nucleus 1246 intersecting exclude region. Skipping ...
Nucleus 1261 intersecting exclude region. Skipping ...
Nucleus 12

/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001207.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001208.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001209.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001211.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/cl

Nucleus 1329 intersecting exclude region. Skipping ...
Nucleus 1331 intersecting exclude region. Skipping ...
Nucleus 1342 intersecting exclude region. Skipping ...
Nucleus 1348 intersecting exclude region. Skipping ...
Nucleus 1354 intersecting exclude region. Skipping ...
Nucleus 1372 intersecting exclude region. Skipping ...
Nucleus 1381 intersecting exclude region. Skipping ...
Nucleus 1384 intersecting exclude region. Skipping ...
Nucleus 1388 intersecting exclude region. Skipping ...
Nucleus 1390 intersecting exclude region. Skipping ...
Nucleus 1395 intersecting exclude region. Skipping ...
Nucleus 1399 intersecting exclude region. Skipping ...
Nucleus 1408 intersecting exclude region. Skipping ...
Nucleus 1421 intersecting exclude region. Skipping ...
Nucleus 1441 intersecting exclude region. Skipping ...
Nucleus 1442 intersecting exclude region. Skipping ...
Nucleus 1447 intersecting exclude region. Skipping ...
Nucleus 1453 intersecting exclude region. Skipping ...
Nucleus 14

/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001465.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001466.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001467.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/classification_border_2/other/WT___CD8 1757 WT___Snap-147723.czi_001468.png is a low contrast image
  io.imsave(png_file_path, im8)
/tmp/ipykernel_240774/2963409519.py:175: UserWarning: /home/chrism/datasets/dapidl/test/tcell/cl

Nucleus 1592 intersecting exclude region. Skipping ...
Nucleus 1593 intersecting exclude region. Skipping ...
Nucleus 1595 intersecting exclude region. Skipping ...
Nucleus 1598 intersecting exclude region. Skipping ...
/home/chrism/datasets/dapidl/test/tcell/cells/HS6ST1___CD8 HS6ST1 12368___Snap-156909_c1+2+3_seg.npy
Nucleus 1 intersecting exclude region. Skipping ...
Nucleus 2 intersecting exclude region. Skipping ...
Nucleus 3 intersecting exclude region. Skipping ...
Nucleus 4 intersecting exclude region. Skipping ...
Nucleus 5 intersecting exclude region. Skipping ...
Nucleus 6 intersecting exclude region. Skipping ...
Nucleus 7 intersecting exclude region. Skipping ...
Nucleus 11 intersecting exclude region. Skipping ...
Nucleus 16 intersecting exclude region. Skipping ...
Nucleus 18 intersecting exclude region. Skipping ...
Nucleus 19 intersecting exclude region. Skipping ...
Nucleus 20 intersecting exclude region. Skipping ...
Nucleus 21 intersecting exclude region. Skipping .